## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2026-02-08-19-01-03 +0000,nypost,California NBA contenders at a cross roads fol...,https://nypost.com/2026/02/08/sports/californi...
1,2026-02-08-18-46-18 +0000,nyt,Breezy Johnson’s Olympic Gold Medal Celebratio...,https://www.nytimes.com/2026/02/08/world/breez...
2,2026-02-08-18-42-48 +0000,nypost,Dem pols battle against national voter ID rule...,https://nypost.com/2026/02/08/us-news/dem-pols...
3,2026-02-08-18-40-20 +0000,nyt,"Starmer’s Chief of Staff Resigns, Citing Role ...",https://www.nytimes.com/2026/02/08/world/europ...
4,2026-02-08-18-38-09 +0000,nypost,Bonehead NJ driver’s stunt gone wrong sends pi...,https://nypost.com/2026/02/08/us-news/bonehead...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2469/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
90,trump,17
69,super,12
188,election,12
70,bowl,12
37,epstein,11


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
80,2026-02-08-12-00-00 +0000,wsj,The Trump economy in 2025 was a roller coaster...,https://www.wsj.com/economy/consumers/american...,50
90,2026-02-08-10-50-00 +0000,startribune,Analysis: Super Bowl GMs show just how off-tar...,https://www.startribune.com/vikings-general-ma...,49
144,2026-02-08-00-12-00 +0000,wsj,Whistleblower complaint against Tulsi Gabbard ...,https://www.wsj.com/politics/national-security...,48
14,2026-02-08-18-15-07 +0000,cbc,What to expect from Bad Bunny’s Super Bowl hal...,https://www.cbc.ca/news/entertainment/super-bo...,48
85,2026-02-08-11-00-00 +0000,latimes,"Trump, California and the multi-front war over...",https://www.latimes.com/politics/story/2026-02...,43


## Select top headlines

In [5]:
with open('exclude_sources.txt') as f:
    excluded_sources = {
        line.strip()
        for line in f
        if line.strip() and not line.strip().startswith('#')
    }

top_count = 10
top_rows = []
working = word_scores.copy()
remaining = latest.copy()

while len(top_rows) < top_count and not remaining.empty:
    scored_remaining = remaining.assign(
        score=remaining['title'].apply(
            lambda t: sum(
                working.get(w.lower(), 0)
                for w in re.findall(r'[A-Za-z]+', t)
                if len(w) > 1
            )
        )
    )
    ranked_loop = scored_remaining.sort_values('score', ascending=False)

    if ranked_loop.empty:
        break

    top_story = ranked_loop.iloc[0]
    remaining = remaining.drop(top_story.name)

    if top_story['source'] in excluded_sources:
        continue

    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for word in words:
        working.pop(word, None)

    top_rows.append(top_story[['score', 'pubdate', 'source', 'title', 'link']])

top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
80,50,2026-02-08-12-00-00 +0000,wsj,The Trump economy in 2025 was a roller coaster...,https://www.wsj.com/economy/consumers/american...
90,49,2026-02-08-10-50-00 +0000,startribune,Analysis: Super Bowl GMs show just how off-tar...,https://www.startribune.com/vikings-general-ma...
50,38,2026-02-08-15-40-07 +0000,nyt,"Savannah Guthrie, in New Video, Promises to Pa...",https://www.nytimes.com/2026/02/07/us/savannah...
27,37,2026-02-08-17-35-40 +0000,nypost,UK PM Keir Starmer’s top aide Morgan McSweeney...,https://nypost.com/2026/02/08/world-news/uk-pm...
39,35,2026-02-08-16-52-15 +0000,bbc,Japan's governing party on course for landslid...,https://www.bbc.com/news/articles/cx2y7d2z29xo...
144,31,2026-02-08-00-12-00 +0000,wsj,Whistleblower complaint against Tulsi Gabbard ...,https://www.wsj.com/politics/national-security...
134,31,2026-02-08-01-01-15 +0000,nyt,Nithya Raman Announces She Will Run for Los An...,https://www.nytimes.com/2026/02/07/us/nithya-r...
21,27,2026-02-08-18-04-45 +0000,nypost,Iran sentences locked-up Nobel Peace Prize win...,https://nypost.com/2026/02/08/world-news/iran-...
5,25,2026-02-08-18-35-44 +0000,startribune,Live: Lindsey Vonn crashes in Olympic downhill...,https://www.startribune.com/2026-winter-olympi...
170,25,2026-02-07-20-48-44 +0000,wapo,"At Winter Olympics, protest over cost and ICE ...",https://www.washingtonpost.com/sports/olympics...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
